# ui

> Fill in a module description here

In [ ]:
#| default_exp ui

In [ ]:
#| hide
#from nbdev.showdoc import *
#!conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.11.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/arthurconner/opt/anaconda3/envs/clue

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.5           |  py311hb9e55a9_0         138 KB
    certifi-2023.11.17         |  py311hecd8cb5_0         160 KB
    intel-openmp-2023.1.0      |   ha357a0b_43548         638 KB
    mkl-2023.1.0               |   h8e150cf_43560       181.0 MB
    mkl-service-2.4.0          |  py311h6c40b1e_1          43 KB
    mkl_fft-1.3.8              |  py311h6c40b1e_0         204 KB
    mkl_random-1.2.4           |  py311ha357a0b_0         272 KB
    numexpr-2.8.7              |  py311h728a8a3_0         162

In [1]:
#| export

from termcolor import colored
from random import shuffle, randint, sample
import emoji
import tabulate
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#| export
import clue.model as model

In [3]:
#| export
def cardFull(info):
    name,color, icon = info[:3]
    if len(icon) > 0:
        ret = icon + " " + colored(name,color)
    else:
        ret = colored(name,color)
    
    return ret

def cardUncolor(info):
    name,color, icon = info[:3]
    if len(icon) > 0:
        ret = icon + " " + name
    else:
        ret = name
    
    return ret

def cardShort(info):
    if len(info) < 3:
        return cardFull(info)
    color,icon, name = info[1:]
    if len(icon) > 0:
        ret = icon + " " + colored(name,color)
    else:
        ret = colored(name,color)
   
    return  ret


In [4]:
#| hide
sampleC = ["hi","red",u"\U0001f460","Scar."]

cardFull(sampleC),cardShort(sampleC)

('👠 hi', '👠 Scar.')

In [5]:
#| export
def cIndentity(x):
  return x[0]
    
def printHand(hand,view=cIndentity):
    container = []
    line = [view(card) for card in hand]
    print(', '.join(line))

def printPlayerHand(player,game):
    hand = [model.keyLookUp(x) for x in model.handOf(player,game)]
    printHand(hand,cardFull)
            


In [6]:
#| hide
#printHand(model.people)
pgame = model.createClue(6)
printPlayerHand(2,pgame)

🧐 Colonel Mustard, Hall, 🥼  Mrs. White


In [7]:
#| hide
printHand(model.people,cardFull)
printHand(model.people,cardShort)

👠 Miss Scarlet, 🧐 Colonel Mustard, 🥼  Mrs. White, 🐉 Mr. Green, 🍆 Professor Plum, 🦚 Mrs. Peacock
👠 Scar., 🧐 Must., 🥼 White, 🐉 Green, 🍆 Plum , 🦚 Peac.


In [8]:
#| export



def printPlayer(game,i):
    ploc = game[-1]
    person = model.people[i-1]
    d = cardFull(person)
    loc = cardFull(model.cardLookup(3,ploc[i-1]))
    d = "[" + str(i) + "] " + d  + " is in " + loc 
    print("\n" + d.center(70," "))
 
    ahand = game[i]
    maps = [model.keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)

    
def printGame(game):
    print("Solution")
    ahand = game[0]
   
    maps = [model.keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)
    for i in range(1,len(game)-1):
        printPlayer(game,i)
        


     
       

In [9]:
#| hide
agame = model.createClue(4)
amoves = []
printGame(agame)
agame[-1], model.rooms

Solution
👠 Miss Scarlet, 🕯️ Candlestick, Study

                  [1] 👠 Miss Scarlet is in Ball Room                  
🐉 Mr. Green, Dining Room, 🔫 Revolver, 🪢 Rope, Library

                   [2] 🧐 Colonel Mustard is in Hall                   
Billiard Room, 🪠 Lead Pipe, Kitchen, 🥼  Mrs. White, Ball Room

                 [3] 🥼  Mrs. White is in Dining Room                  
🧐 Colonel Mustard, 🔧 Wrench, Lounge, 🍆 Professor Plum

                     [4] 🐉 Mr. Green is in Study                      
Hall, Conservatory, 🦚 Mrs. Peacock, 🔪 Knife


([0, 4, 3, 8, 7, 6],
 [['Ball Room', 'cyan', '', 'Ball'],
  ['Billiard Room', 'cyan', '', 'Bill'],
  ['Conservatory', 'cyan', '', 'Consv'],
  ['Dining Room', 'cyan', '', 'Dining'],
  ['Hall', 'cyan', '', 'Hall'],
  ['Kitchen', 'cyan', '', 'Kitch'],
  ['Lounge', 'cyan', '', 'Lounge'],
  ['Library', 'cyan', '', 'Lib'],
  ['Study', 'cyan', '', 'Study']])

In [ ]:
#| hide


## Response and Answer
    
        


In [11]:
#| export
def showChoice(guess):
    p,w,l = guess[:3]
    ret = cardFull(model.cardLookup(1,p))
    ret = ret + " with the " + cardFull(model.cardLookup(2,w))
    ret = ret + " in the " + cardFull(model.cardLookup(3,l))
    return ret
    

In [12]:
#| hide
aguess = (0,2,7)
print(showChoice(aguess))

👠 Miss Scarlet with the 🪢 Rope in the Library


In [14]:
#| hide
aresponse =  model.responseToGuess(aguess,1,4, model.keysForGuess(aguess)[0])
aresponse, showChoice(aresponse)

((0, 2, 7, 1, 4, 100), '👠 Miss Scarlet with the 🪢 Rope in the Library')

In [15]:
#| hide
amoves.append(aresponse)

In [52]:
#| hide
amoves=[]
for x in range(90):
    pc = randint(1,4)
    p = randint(0,5)
    w =  randint(0,5)
    l = randint(0,5)
    aguess = (p,w,l)
    aresponse =  model.getAnswer(pc,agame,aguess)
    if aresponse[0] > -1:
        amoves.append(aresponse)


        

amoves

[(1, 4, 5, 1, 2, 305),
 (5, 3, 3, 3, 4, 105),
 (1, 2, 0, 1, 2, 300),
 (1, 2, 0, 3, 1, 202),
 (1, 4, 0, 2, 3, 101),
 (4, 1, 5, 3, 1, 201),
 (0, 0, 5, 1, 2, 305),
 (1, 3, 4, 2, 3, 101),
 (3, 2, 2, 3, 4, 302),
 (3, 5, 4, 3, 4, 304),
 (0, 3, 3, 2, 3, 203),
 (1, 1, 2, 3, 4, 302),
 (5, 2, 2, 4, 1, 202),
 (1, 4, 1, 4, 2, 301),
 (0, 3, 5, 3, 2, 305),
 (1, 4, 2, 2, 3, 101),
 (1, 5, 0, 1, 2, 205),
 (1, 3, 2, 1, 3, 101),
 (3, 5, 1, 4, 1, 103),
 (1, 0, 3, 4, 1, 303),
 (1, 2, 5, 4, 1, 202),
 (3, 3, 1, 2, 3, 203),
 (4, 3, 3, 2, 3, 104),
 (2, 1, 1, 3, 1, 201),
 (1, 2, 5, 1, 2, 305),
 (0, 0, 3, 3, 4, 200),
 (1, 1, 5, 1, 2, 305),
 (3, 4, 2, 3, 4, 302),
 (3, 0, 1, 2, 4, 200),
 (0, 2, 0, 4, 1, 202),
 (4, 1, 3, 2, 3, 104),
 (3, 4, 5, 2, 1, 103),
 (0, 2, 2, 3, 4, 302),
 (0, 3, 0, 1, 2, 300),
 (3, 4, 0, 4, 1, 103),
 (4, 1, 5, 3, 1, 201),
 (4, 5, 3, 3, 1, 303),
 (0, 3, 3, 4, 1, 303),
 (0, 0, 3, 3, 4, 200),
 (4, 5, 2, 1, 2, 205),
 (1, 2, 4, 2, 3, 101),
 (4, 2, 5, 1, 2, 305),
 (5, 1, 0, 2, 4, 105),
 (5, 2, 4, 

In [66]:
#| export
def wrapL(s,f,l):
    show = set([s-1])
    s += 1
    if s > l:
        s = 1
    while s != f:
        show.add(s-1)
        s += 1
        if s > l:
            s = 1
    show.add(f-1)
    
    ret = ""
    for x in range(l):
        if x in show:
            ret += model.people[x][2]
        else:
            ret += "❓" 
    return ret
        
def showMoves(pc,moves,plMax):
    for answer in moves:
        p,w,l,asker, answered,shown = answer
        line = model.people[asker-1][2] + "  asks "
        line += "(" + model.people[p][2] + " &  " + model.weapons[w][2] + " in the "       
        line += ( model.rooms[l][0] +") ").ljust(15,"-")
        line += "->" + wrapL(asker,answered,plMax)
        if asker == pc or pc < 0 :
            k =  int(shown/100)
            s = shown%100
            line += " shows " + cardFull(model.cardLookup(k,s))
        print(line)

        if False:
            
            print(showChoice(answer))
            print(answer)
            print()
        
#showMoves(1,amoves,4)     
        
def showPasses(icon,moves,game):
    myMoves = [ x for x in moves if x[-1] == icon]
    ret = set()
    for x in myMoves:
        g = model.passedOnResponse(x,game)
        print(g)
        for y in g:
            ret.add(y)
    p = len(game) - 2
    showMoves(-1,myMoves,p )
    #g = model.passedOnResponse(ans,agame)
    return y

In [69]:
#| hide
#showMoves(-1,amoves,4)
showPasses(203,amoves,agame)

[]
[]
[]
[]
[]
[1]
🧐  asks (👠 &  🔧 in the Dining Room) --->❓🧐🥼❓ shows 🔧 Wrench
🧐  asks (🐉 &  🔧 in the Billiard Room) ->❓🧐🥼❓ shows 🔧 Wrench
🧐  asks (🐉 &  🔧 in the Dining Room) --->❓🧐🥼❓ shows 🔧 Wrench
🧐  asks (🧐 &  🔧 in the Conservatory) -->❓🧐🥼❓ shows 🔧 Wrench
🧐  asks (🥼 &  🔧 in the Billiard Room) ->❓🧐🥼❓ shows 🔧 Wrench
👠  asks (👠 &  🔧 in the Dining Room) --->👠🧐🥼❓ shows 🔧 Wrench


1

In [58]:
g = model.passedOnResponse(200,agame)

TypeError: 'int' object is not subscriptable

In [ ]:
emoji.emojize(":question:"),u"\U0001F97C"

(':question:', '🥼')

In [ ]:
#| hide
model.getSeen(1,agame,amoves)

[301, 103, 203, 104, 200, 304]

In [ ]:
aresponse

(0, 2, 7, 1, 4, 100)

In [ ]:
#| hide
g = model.passedOnResponse(aresponse,agame)

In [ ]:
#| export
def iconOfPlayers(pList):
    pList.sort()
    return ", ".join([model.people[x-1][2] for x in pList if x>0] )
    

In [ ]:
#| hide
iconOfPlayers(g)

'🍆'

In [ ]:
b = model.notInHands(agame,amoves)


In [ ]:
print(amoves)

[(3, 0, 4, 3, 1, 200), (2, 5, 4, 1, 2, 304), (5, 1, 2, 4, 2, 105), (4, 1, 2, 2, 3, 302), (4, 5, 0, 4, 1, 104), (4, 4, 4, 2, 1, 104), (1, 0, 0, 4, 1, 200)]


## Input

In [ ]:
def chooseCard(hand,view=cIndentity,showN=True,flag = {}):
    container = []
    i = 1
    ml = max([len(x[0]) for x in hand])

    no = " "
    
    for card in hand:
        face = view(card)
        #face.append(i)
        color = card[1]
        no = " "
        if card[0] in flag:
            no = "*"
        if showN:
            preamb = str(i) + ")"
        else:
            preamb = ""
        container.append(colored( preamb + no + " " ,color) + face)
        i += 1
    choice = -1
    
    while choice < 1:
        for card in container:
            print(card)
        choice = int(input("Choose from 1 to " +  str(len(container))+": "))
       
        if (choice > 0) and (choice <= len(container)):
            return choice
        print(str(choice) + " is not in range")
        choice = -1

In [ ]:
#| hide
chooseCard(model.people,cardFull,flag = {"Mr. Green"})

1)  👠 Miss Scarlet
2)  🧐 Colonel Mustard
3)  ♡  Mrs. White
4)* 🐉 Mr. Green
5)  🍆 Professor Plum
6)  🦚 Mrs. Peacock


Choose from 1 to 6:  4


4

In [ ]:
#| hide
chooseCard(model.people,cardShort,False,flag = {"Mr. Green"})

  👠 Scar.
  🧐 Must.
  ♡ White
* 🐉 Green
  🍆 Plum 
  🦚 Peac.


Choose from 1 to 6:  4


4

In [ ]:
#| export
def getChoices(keys,playerID,game,moves,showIndex):

    container = []
    i = 1
    #ml = max([len(x[0]) for x in hand])

    passedOn = model.notInHands(game,moves)
    flagged = set(model.getSeen(playerID, game, moves))
    ret = []
    for pair in keys:
        no = " "
        key = model.keyForArray(pair)
    
        if key in flagged:
            no = "*"
        data = model.cardForArray(pair)
        view = cardFull(data)
        if showIndex:
            preamble = str(i)
        else:
            preamble = ""
            #view = cardUncolor(data)
        base =  preamble + no + view
        if key in passedOn:
            shownTo = passedOn[key]
            base = base.ljust(35)  + iconOfPlayers(list(shownTo))
            
        ret.append(base)
        i+=1
    return ret
        
    


    
def chooseKey(keys,playerID,game,moves):
    i = 1
    #ml = max([len(x[0]) for x in hand])

    passedOn = model.notInHands(game,moves)
    flagged = set(model.getSeen(playerID, game, moves))
    keyChoice = -1
    lines = "\n".join(getChoices(keys,playerID,game,moves,True))

    while keyChoice < 1:   
        print(lines)
        keyChoice = int(input("Choose from 1 to " +  str(len(keys))+": "))
        
        if (keyChoice > 0) and (keyChoice <= len(keys)):
            return keyChoice
        print(str(keyChoice) + " is not in range")
        keyChoice = -1
    return keyChoice
    

In [ ]:
wKeys = [[1,x] for x in range(len(model.people))]
chooseKey(wKeys,1,agame,amoves)

1 👠 Miss Scarlet
2 🧐 Colonel Mustard
3 ♡  Mrs. White
4*🐉 Mr. Green
5*🍆 Professor Plum        ♡, 🐉, 🍆
6 🦚 Mrs. Peacock          👠, 🍆


Choose from 1 to 6:  3


3

In [ ]:
wKeys = [[1,x] for x in range(len(model.people))]
chooseKey(wKeys,2,agame,amoves)

1*👠 Miss Scarlet
2*🧐 Colonel Mustard
3 ♡  Mrs. White
4 🐉 Mr. Green
5*🍆 Professor Plum        ♡, 🐉, 🍆
6*🦚 Mrs. Peacock          👠, 🍆


Choose from 1 to 6:  2


2

In [ ]:
#| export
def showMultipleKey(keys,headers,playerID,game,moves):
    d = {}
    for i in range(len(keys)):
        col = getChoices(keys[i],playerID,game,moves,False)
        d[headers[i]] = col
        
    
    dataf = pd.DataFrame.from_dict(d)
    dataf.index = np.arange(1, len(dataf) + 1)
    return dataf
    


    
def choosePandW(playerID,game,moves):
    i = 1
    #ml = max([len(x[0]) for x in hand])

    pKeys = [[1,x] for x in range(len(model.people))]
    wKeys = [[2,x] for x in range(len(model.weapons))]
    df = showMultipleKey([pKeys,wKeys],["People","Weapons"],playerID,game,moves)

    

    print(tabulate.tabulate(df, showindex=True, headers=df.columns))
    keyChoice, y = [int(x) for x in input().split()] 
    return keyChoice, y
        

In [ ]:
#hide
choosePandW(2,agame,amoves)

    People                                Weapons
--  ------------------------------------  --------------------------------
 1  *👠 Miss Scarlet                      🔪 Knife                  🐉, 🍆
 2  *🧐 Colonel Mustard                   🔫 Revolver
 3  ♡  Mrs. White                         🪢 Rope
 4  🐉 Mr. Green                          🔧 Wrench
 5  *🍆 Professor Plum         ♡, 🐉, 🍆  *🕯️ Candlestick
 6  *🦚 Mrs. Peacock           👠, 🍆     🪠 Lead Pipe


 3 4


(3, 4)

## board mechanics

In [ ]:
#| export
def playerMarker(i):
    info = model.people[i]
    color,icon, name = info[1:]

    return colored(name[0],color)
   
    


In [ ]:
print(playerMarker(0))

S


In [ ]:
def notInHands(game,moves):
    ret = dict()
    
    for x in moves:
        
        
        thePasses = passedOnResponse(x,game) 
        
        if len(thePasses) > 0:
            shown = x[-1]
            if shown in ret:
                addToMe = ret[shown]
            else:
                addToMe = set()
            for aPass in thePasses:
                addToMe.add(aPass)
            ret[shown] = addToMe
                
       
    return ret

In [ ]:
#| export
def roomFor(i,ch,where,visited):
    #print(i)
 
    if i<1:
        return ""
    adds = ""
    
    peeps = ", ".join([playerMarker(x) for x in where[i-1]])

    try:
        x = ch.index(i-1)
        adds = str(x+1) +") "
    except:
        adds = ""
        #print(i)

    roomName = model.rooms[i-1][0]
    k = model.keyCard(3,i-1)
    
    if k in visited:
        roomName = colored(roomName,"red")
        

    
    return adds + roomName +"\n   " + peeps
    
def showBoard(pc,game,moves):
    roomCount = len(model.rooms)
    
    where = [[] for x in range(roomCount)]
    ploc = game[-1]
    #print(ploc)
    for i in range(len(ploc)):
        suspect = i + 1
        room = model.roomOf(suspect,game)
        where[room].append(i)

    visited  = set(model.getSeen(pc,game, moves))
    
    lines = [[1,2,3],[9,0,4],[8,0,5],[0,7,6]]
    
    playerRoom = model.roomOf(pc,game)
    b = model.roomsAround(playerRoom)

    m = []

    for x in lines:
        m.append([roomFor(i,b,where,visited) for i in x])
    print(tabulate.tabulate(m,  tablefmt="fancy_grid"))

In [ ]:
showBoard(1,agame,amoves)

╒══════════════╤═══════════════╤══════════════╕
│ 3) Ball Room │ Billiard Room │ Conservatory │
│              │    M          │    W         │
├──────────────┼───────────────┼──────────────┤
│ 2) Study     │               │ Dining Room  │
│    S         │               │    P         │
├──────────────┼───────────────┼──────────────┤
│ 1) Library   │               │ Hall         │
│              │               │    G         │
├──────────────┼───────────────┼──────────────┤
│              │ Lounge        │ Kitchen      │
│              │               │    P         │
╘══════════════╧═══════════════╧══════════════╛


In [ ]:
#| export
def showLocations(game):
    ploc = game[-1]

    for i in range(len(ploc)):
        pc = i + 1
        person = model.people[i]
        d = cardFull(person)
        room = model.roomOf(pc,game)
        loc = cardFull(model.cardLookup(3,room))
        d = "[" + str(i+1) + "] " + d  + " is in " + loc 
        print(d)

In [ ]:
#| hide
showLocations(agame)
printGame(agame)

[1] 👠 Miss Scarlet is in Study
[2] 🧐 Colonel Mustard is in Billiard Room
[3] ♡  Mrs. White is in Conservatory
[4] 🐉 Mr. Green is in Hall
[5] 🍆 Professor Plum is in Kitchen
[6] 🦚 Mrs. Peacock is in Dining Room
Solution
🦚 Mrs. Peacock, 🪢 Rope, Billiard Room

           [1] 👠 Miss Scarlet is in Study           
Library, 🍆 Professor Plum, 🔪 Knife, Dining Room, 🔧 Wrench

     [2] 🧐 Colonel Mustard is in Billiard Room      
Ball Room, Conservatory, Lounge, 🪠 Lead Pipe, Hall

        [3] ♡  Mrs. White is in Conservatory        
👠 Miss Scarlet, Kitchen, 🐉 Mr. Green, 🕯️ Candlestick

             [4] 🐉 Mr. Green is in Hall             
🔫 Revolver, ♡  Mrs. White, Study, 🧐 Colonel Mustard


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()